In [12]:
!python3 -m pip install clickhouse-driver --user

     |████████████████████████████████| 150 kB 2.5 MB/s eta 0:00:01


In [96]:
import math


class FibonacciHeap_for_vertex:

    # internal node class
    class Node:
        def __init__(self, key, value):
            self.key = key
            self.value = value
            self.parent = self.child = self.left = self.right = None
            self.degree = 0
            self.mark = False

    # function to iterate through a doubly linked list
    def iterate(self, head):
        node = stop = head
        flag = False
        while True:
            if node == stop and flag is True:
                break
            elif node == stop:
                flag = True
            yield node
            node = node.right

    # pointer to the head and minimum node in the root list
    root_list, min_node = None, None

    # maintain total node count in full fibonacci heap
    total_nodes = 0

    # return min node in O(1) time
    def find_min(self):
        return self.min_node

    # extract (delete) the min node from the heap in O(log n) time
    # amortized cost analysis can be found here (http://bit.ly/1ow1Clm)
    def extract_min(self):
        z = self.min_node
        if z is not None:
            if z.child is not None:
                # attach child nodes to root list
                children = [x for x in self.iterate(z.child)]
                for i in range(0, len(children)):
                    self.merge_with_root_list(children[i])
                    children[i].parent = None
            self.remove_from_root_list(z)
            # set new min node in heap
            if z == z.right:
                self.min_node = self.root_list = None
            else:
                self.min_node = z.right
                self.consolidate()
            self.total_nodes -= 1
        return z

    # insert new node into the unordered root list in O(1) time
    # returns the node so that it can be used for decrease_key later
    def insert(self, key, value=None):
        n = self.Node(key, value)
        n.left = n.right = n
        self.merge_with_root_list(n)
        if self.min_node is None or n.key < self.min_node.key:
            self.min_node = n
        self.total_nodes += 1
        return n

    # modify the key of some node in the heap in O(1) time
    def decrease_key(self, x, k):
        if k > x.key:
            return None
        x.key = k
        y = x.parent
        if y is not None and x.key < y.key:
            self.cut(x, y)
            self.cascading_cut(y)
        if x.key < self.min_node.key:
            self.min_node = x

    # merge two fibonacci heaps in O(1) time by concatenating the root lists
    # the root of the new root list becomes equal to the first list and the second
    # list is simply appended to the end (then the proper min node is determined)
    def merge(self, h2):
        H = FibonacciHeap()
        H.root_list, H.min_node = self.root_list, self.min_node
        # fix pointers when merging the two heaps
        last = h2.root_list.left
        h2.root_list.left = H.root_list.left
        H.root_list.left.right = h2.root_list
        H.root_list.left = last
        H.root_list.left.right = H.root_list
        # update min node if needed
        if h2.min_node.key < H.min_node.key:
            H.min_node = h2.min_node
        # update total nodes
        H.total_nodes = self.total_nodes + h2.total_nodes
        return H

    # if a child node becomes smaller than its parent node we
    # cut this child node off and bring it up to the root list
    def cut(self, x, y):
        self.remove_from_child_list(y, x)
        y.degree -= 1
        self.merge_with_root_list(x)
        x.parent = None
        x.mark = False

    # cascading cut of parent node to obtain good time bounds
    def cascading_cut(self, y):
        z = y.parent
        if z is not None:
            if y.mark is False:
                y.mark = True
            else:
                self.cut(y, z)
                self.cascading_cut(z)

    # combine root nodes of equal degree to consolidate the heap
    # by creating a list of unordered binomial trees
    def consolidate(self):
        A = [None] * int(math.log(self.total_nodes) * 2)
        nodes = [w for w in self.iterate(self.root_list)]
        for w in range(0, len(nodes)):
            x = nodes[w]
            d = x.degree
            while A[d] != None:
                y = A[d]
                if x.key > y.key:
                    temp = x
                    x, y = y, temp
                self.heap_link(y, x)
                A[d] = None
                d += 1
            A[d] = x
        # find new min node - no need to reconstruct new root list below
        # because root list was iteratively changing as we were moving
        # nodes around in the above loop
        for i in range(0, len(A)):
            if A[i] is not None:
                if A[i].key < self.min_node.key:
                    self.min_node = A[i]

    # actual linking of one node to another in the root list
    # while also updating the child linked list
    def heap_link(self, y, x):
        self.remove_from_root_list(y)
        y.left = y.right = y
        self.merge_with_child_list(x, y)
        x.degree += 1
        y.parent = x
        y.mark = False

    # merge a node with the doubly linked root list
    def merge_with_root_list(self, node):
        if self.root_list is None:
            self.root_list = node
        else:
            node.right = self.root_list.right
            node.left = self.root_list
            self.root_list.right.left = node
            self.root_list.right = node

    # merge a node with the doubly linked child list of a root node
    def merge_with_child_list(self, parent, node):
        if parent.child is None:
            parent.child = node
        else:
            node.right = parent.child.right
            node.left = parent.child
            parent.child.right.left = node
            parent.child.right = node

    # remove a node from the doubly linked root list
    def remove_from_root_list(self, node):
        if node == self.root_list:
            self.root_list = node.right
        node.left.right = node.right
        node.right.left = node.left

    # remove a node from the doubly linked child list
    def remove_from_child_list(self, parent, node):
        if parent.child == parent.child.right:
            parent.child = None
        elif parent.child == node:
            parent.child = node.right
            node.right.parent = parent
        node.left.right = node.right
        node.right.left = node.left

In [32]:
class MapWay:

    def __init__(self,
                 nameAlgo=None,
                 arrAsks=None,
                 current_position=None,
                 points=None,
                 model=None,
                 places=None):
        import math
        self.nameAlgo = nameAlgo
        self.arrAsks = arrAsks
        self.points = points
        self.model = model
        self.places = places
        self.infinity = math.inf

    def findBestWay(self, user=None, points=None):
        '''
        user: пользователь для которого мы анализируем
        points: пока что массив на две точки
        [0]: начальная
        [1]: конечная
        '''
        import math
        from collections import deque
        # поиск объектов, которые попадают под интересующую нас зону
        
        '''
            Переделать всё это в один SQL запрос
            чтобы не ханить все точки в памяти и
            вычисления проводились на сервере
        
        '''
#         places_in_zone = []
#         for place in self.places: 
#             if self.inZone(first_point=points[0],
#                            second_point=points[1],
#                            object_place=place):
#                 places_in_zone.append(place)

        '''
            Добавление начальной и конечной точки,
            можно будет добавить промежуточную
        '''
        places_in_zone = self.inZone(points[0], points[1])
        for point in points:
            places_in_zone.append(point)
        '''
        Построение графа исходя из модели

        1) Возьмём и посмотрим насколько подходит пользователю каждое место из предложенных,
            для этого будем использовать ранг каждого места для посетителя, рачитывать его мы будем по формуле,
            { Формулу я ещё не придумал }

        '''
        self.matrix_size_x = len(places_in_zone)
        matrix_places = [[0] * self.matrix_size_x for _ in range(self.matrix_size_x)]

        for i in range(self.matrix_size_x):
            for j in range(self.matrix_size_x):
                '''
                    Добаляю расстояния в матрицу

                    !!!!  Здесь нам надо будет брать время,
                    которое понадобится для прохода из
                    одной точки в другую с яндекс карт  !!!!

                    Расстояние думаю лучше измерять во времени, которое человек затратит на путь
                '''
                if (((places_in_zone[i].category == 'Start_point' and places_in_zone[j] == 'End_point') or
                     (places_in_zone[i].category == 'End_point' and places_in_zone[j] == 'Start_point')) and
                        self.matrix_size_x > 2):
                    matrix_places[i][j] = math.inf
                else:
                    matrix_places[i][j] = self.getLength(places_in_zone[i], places_in_zone[j])

        '''

        S
         \
          \
           \
            *-------E

        Буду использовать алгоритм дейкстры для поиска пути, но буду так же искать по точке,
        которая больше всего нравится пользователю и по времени до конца
        '''

        start_point = len(matrix_places) - len(points)
        end_point = len(matrix_places) - 1
        dist = [math.inf] * len(matrix_places)
        way = [0] * len(matrix_places)
        way[start_point] = 0
        dist[0] = 0
        Q = deque()
        Q.append(len(places_in_zone) - len(points))
        while Q:
            v = Q.pop()
            for u in range(len(matrix_places[v])):
                if (dist[u] > dist[v] + matrix_places[v][u]):
                    dist[u] = dist[v] + matrix_places[v][u]
                    way[u] = v
                    Q.append(u)

        '''

                Чтобы восстановить оптимальный путь от
                начальной вершины до коненой нам нужно просто смотреть на массив
                в котором у нас записаны минимальные расстояния до начальной вершины
                Мы будем жадно брать самое маленькое значение

        '''

        road = [end_point]
        now_point = end_point
        while way[now_point] != now_point:
            now_point = way[end_point]
            road.append(now_point)
        
        ans = [points[0]]
        for i in list(reversed(road)):
            ans.append(places_in_zone[i])
        return ans

    
    def all_pinned_points(self, 
                          points = None, 
                          start = None, 
                          end = None):
        '''
            TODO: Закэшировать все расстояния между объектами, чтобы потом не делать запросы на сервера
            использую алгоритм построения минимального остового дерева (Алгоритм Прима)
        '''
        import math
        import heapq
        import random
        way = [start, end]
        if points == None:
            return way
        way.append(points)
        
        matrix = [[math.inf] for i in range(len(way))]
        
        for i in range(len(matrix)):
            for j in range(len(matrix)):
                matrix[i][j] = getLength(())
                
        key = [math.inf] * len(way)
        p = [None] * len(way)
        r = random.randint(0, len(matrix))
        key[r] = 0
        
        Q = way
        heapq.heapify(Q)
        
            
    
    def inZone(self,
               first_point=None,
               second_point=None):
        '''
            TODO: добавить функцию просмотра в окружении на точках, для захвата большего количества возможных мест

            y                                         y
            ^                                         ^
            X----------------------                   --------------------Y
            |         *           |                   |   *               |   *
            |                     | *            *    |            *      |
            |    *         *      |                   |                   |
            |                     |                   |       *           |
        *   |                     |                   |                   |
            |         *       *   |    *              |                   | *
            |                     |               *   |   *               |
          * ----------------------Y > x               X-------------------- > x

        '''
        left_point = None
        right_point = None
        
        # Problem: make all this with one query
        
        
        '''
            У нас есть эти точки изначально, поэтому можно вычислить их положение
        '''
        if first_point.position.longtitude <= second_point.position.longtitude: # 
            left_point = first_point
            right_point = second_point
        else: #
            left_point = second_point
            right_point = first_point

        '''
            Сравнения с этими точками можно переделать польностью в один запрос
            
            
            if (left_point.position.latitude >= right_point.position.latitude) and (object_place.position.longtitude <= right_point.position.longtitude and
                    object_place.position.longtitude >= left_point.position.longtitude and
                    object_place.position.latitude <= left_point.position.latitude and
                    object_place.position.latitude >= right_point.position.latitude):
                return True
            else:
                return False
                
            if left_point.position.latitude < right_point.position.latitude and (object_place.position.longtitude <= right_point.position.longtitude and
                    object_place.position.longtitude >= left_point.position.longtitude and
                    object_place.position.latitude <= right_point.position.latitude and
                    object_place.position.latitude >= left_point.position.latitude):
                return True
            else:
                return False
            
            if (left_point.position.latitude >= right_point.position.latitude) and (object_place.position.longtitude <= right_point.position.longtitude and
                    object_place.position.longtitude >= left_point.position.longtitude and
                    object_place.position.latitude <= left_point.position.latitude and
                    object_place.position.latitude >= right_point.position.latitude) and 
                    left_point.position.latitude < right_point.position.latitude and (object_place.position.longtitude <= right_point.position.longtitude and
                    object_place.position.longtitude >= left_point.position.longtitude and
                    object_place.position.latitude <= right_point.position.latitude and
                    object_place.position.latitude >= left_point.position.latitude):
                return True
            else:
                return False
                
            
            Всё преобразовано в один запрос
            
            "SELECT * FROM Place WHERE ({left_point.position.latitude} >= {right_point.position.latitude} and 
                (Place.longtitude <= {right_point.position.longtitude} and
                    Place.longtitude >= {left_point.position.longtitude} and
                    Place.latitude <= {left_point.position.latitude} and
                    Place.latitude >= {right_point.position.latitude)}) or 
                    ({left_point.position.latitude} < {right_point.position.latitude} and 
                    (Place.longtitude <= {right_point.position.longtitude} and
                    Place.longtitude >= {left_point.position.longtitude} and
                    Place.latitude <= {right_point.position.latitude} and
                    Place.latitude >= {left_point.position.latitude)});"
                    
            if left_point.position.latitude >= right_point.position.latitude and (object_place.position.longtitude <= right_point.position.longtitude and
                    object_place.position.longtitude >= left_point.position.longtitude and
                    object_place.position.latitude <= left_point.position.latitude and
                    object_place.position.latitude >= right_point.position.latitude) # 1 
                return True
            else:
                return False
                
            if left_point.position.latitude < right_point.position.latitude and (object_place.position.longtitude <= right_point.position.longtitude and
                    object_place.position.longtitude >= left_point.position.longtitude and
                    object_place.position.latitude <= right_point.position.latitude and
                    object_place.position.latitude >= left_point.position.latitude)
                return True
            else:
                return False
                    
        '''
        query = f"SELECT * FROM Place WHERE ({left_point.position.latitude} >= {right_point.position.latitude} and (Place.longtitude <= {right_point.position.longtitude} and (Place.longtitude >= {left_point.position.longtitude} and Place.latitude <= {left_point.position.latitude} and Place.latitude >= {right_point.position.latitude})) or ({left_point.position.latitude} < {right_point.position.latitude} and (Place.longtitude <= {right_point.position.longtitude} and Place.longtitude >= {left_point.position.longtitude} and Place.latitude <= {right_point.position.latitude} and Place.latitude >= {left_point.position.latitude})));"

        mycursor = mydb.cursor()
        mycursor.execute(query)
        places = mycursor.fetchall()
        places_in_zone = []
        for x in places:
            place = Place(name=x[1], category=x[2],position=Position(latitude=x[3], longtitude=x[4]))
            places_in_zone.append(place)
        return places_in_zone

    def getLength(self, first_point, second_point):
        '''
        :param first_point:
        :param second_point:
        Брать
        :return:
        '''
        import math
        return math.sqrt((first_point.position.latitude - second_point.position.latitude) ** 2 +
                         (first_point.position.longtitude - second_point.position.longtitude) ** 2)

    '''
    Ниже я буду пытаться сам написать алгоритм
    поиска пути исходя из наших критериев поиска
    а именно предпочтений пользователя и времени
    '''

    def evg_algo(self):
        '''

            Надо взять как минимум одну достопримечательность + (Сделал бесконечное расстояние до конечной точки)
            если она имеется в предложенной области
            и смотреть на время, которое указал пользователь при вводе
            при этом надо взять самые интересные, поэтому мы должны
            отсортировать объекты по степени их интересности и смотреть
            от самого интересного к менее интересному

            ----------------------------------
            |                                |
            |     S----\                     |
            |          -[]                   |
            |                                |
            |     []           []            |
            |                       []       |
            |                                |
            |            []                  |
            |                         E      |
            |                                |
            ----------------------------------
        '''

        pass

    '''
    Ниже представлены алгоритм Куна, думал его можно как-то
    использовать для поиска удовлетворяющего пути,
    но не думаю что он будет эффективен
    '''
    # def kunAlgo(self):
    #     '''
    #     Здесь тоже ошибки, потому что алгоритм написан для общего
    #     случая, надо переделать под нашу ситуацию
    #     '''
    #     fill(matching, -1)
    #     for i in range(n):
    #         fill(used, False)
    #         dfs(i)
    #     for i in range(n):
    #         if(matching[i] != - 1):
    #             print(i + " " + matching[i])
    #     pass
    #
    # def fill(self, graph, number):
    #     '''
    #     Для заполнения матрицы каким-то одним значением
    #     '''
    #     for i in range(len(graph)):
    #         for j in range(len(graph[i])):
    #             graph[i][j] = number

    # def dfs(self, v:int):
    #     '''
    #     Здесь ошибка
    #     '''
    #     if (used[v]):
    #         return False
    #     used[v] = True
    #     for to in g[v]:
    #         if (matching[to] == -1 or dfs(matching[to])): #ошбика где dfs(matching[to])
    #             matching[to] = v
    #             return True
    #     return False


class WorkWithDB:

    def __init__(self):
        pass


class User:

    def __init__(self,
                 time=None,
                 age=None,
                 religion=None,
                 sex=None,
                 marital_status=None,
                 other_info=None):
        '''
            Create or get dataset
            Сделал все метрики для лучшего понимания что собирать с людей
        '''

    #         self.category = model.predict()

    def getCategory(self):
        return self.category


class GraphType:

    def __init__(self,
                 length=None,
                 vertex_number=None):
        self.length = length
        self.vertex_number = vertex_number


class Position:

    def __init__(self,
                 latitude=None,
                 longtitude=None,
                 type_of_point=None):
        self.latitude = latitude # Широта
        self.longtitude = longtitude # Долгота


class Place:
    '''
        ---------------------------------------------
        |                                           |
        |   Категории мест:                         |
        |       Start_point: начальная точка        |
        |           пути                            |
        |       Middle_point: точки, которые        |
        |           пользователь отметил между      |
        |           начальной и конечной            |
        |       End_point: конечная точка пути      |
        |       Place_point: точка с местом         |
        |                                           |
        |                                           |
        ---------------------------------------------
    '''

    def __init__(self,
                 name=None,
                 position: Position = None,
                 category=None,
                 rank=None,
                 length=None,
                 visit_time=None):
        self.name = name
        self.position = position
        self.category = category
        self.length = length
        self.rank = rank
        self.visit_time = visit_time


class Metrics_and_algos:

    def __init__(self):
        pass

    def confustion_matrix(self):
        pass

    def accuracy_metric(self):
        '''
        Эту метрику можно назвать базовой.
        Она измеряет количество верно классифицированных объектов
        относительно общего количества всех объектов.
        '''
        pass

    def sensitivity_metric(self):
        '''
        Сколько объектов наша модель смогла правильно классифицировать
        с позитивной меткой из всего множества позитивных.
        '''
        pass

    def precision_metric(self):
        '''
        Сколько из всех объектов, которые классифицируются как положительные,
        действительно являются положительными, относительно общего
        количества полученных от модели позитивных меток.
        '''
        pass

    def f1_score(self):
        '''
        Сочетание precision и recall,
        дает некоторый компромисс между ними двумя,
        оценка F1 достигает своего наилучшего значения в 1 и худшее в 0.
        '''
        pass


In [49]:
a = [3,3,3,32,1,2]
b = [5,3,1]
for i in b:
    a.append(i)

def fill(graph, number):
    for i in range(len(graph)):
        for j in range(len(graph[i])):
            graph[i][j] = number
            
graph = [
    [2, 4 ,3],
    [5, 1, 7],
    [6, 3, 1]
]
fill(graph, -1)
graph

a = [3,3,3,32,1,2]
a.reverse()
a

for i in range(2, len(a)):
    print(a[i])

32
3
3
3


In [43]:
'''
    -----------------------
    |                     
    |                    
    |                    
    |                     
    |
    
    
p1        0.542147 0.7543 - Начальная точка
p2        0.543147 0.7843 - вне первого
p3        0.541147 0.7513 
p4        0.542147 0.7543
p5        0.542147 0.7543
p6        0.542147 0.7543
p7        0.542147 0.7543
p8        0.512147 0.8543 - Конечная точка

s =(20, 2)

p_in_1 = (10, 50)
p_in_2 = (18, 70)
p_out_1 = (21, 135)
p_out_2 = (-10, -30)

e = (5, 100)


'''

# set_class

# set_points
start = Place(name="Start", position=Position(latitude=20, longtitude=2))

p_in_1 = Place(name="p_in_1", position=Position(latitude=10, longtitude=50))
p_in_2 = Place(name="p_in_2", position=Position(latitude=18, longtitude=70))
p_out_1 = Place(name="p_out_1", position=Position(latitude=21, longtitude=135))
p_out_2 = Place(name="p_out_2", position=Position(latitude=-10, longtitude=-30))

end = Place(name="End", position=Position(latitude=5, longtitude=100))

places = [p_in_1, p_in_2, p_out_1, p_out_2]
points = [start, end]

algo = MapWay(places=places)
# find_way
# algo.findBestWay(points=points)
route = list(algo.findBestWay(points=points))
for place in route:
    print(place.name)

Start
p_in_1
End


In [17]:
import mysql.connector

mydb = mysql.connector.connect(
    host='std-mysql',
    user='std_1455_map_way',
    passwd='12345678',
    database='std_1455_map_way'
)

mycursor = mydb.cursor()
mycursor.execute("SELECT * FROM Place;")
myresult = mycursor.fetchall()
print(myresult[0])

def toPlace(places):
    arr = []
    for x in places:
        place = Place(name=x[1], category=x[2],position=Position(latitude=x[3], longtitude=x[4]))
        arr.append(place)
    return arr
all_places = toPlace(myresult)
for place in all_places:
    print(place.name)

(1, 'p_in_1', 2, 10.0, 50.0)
p_in_1
p_in_2
p_out_1
p_out_2


In [29]:
### Сделаю условный запрос для быстроты

first_point = Place(name="Start", position=Position(latitude=20, longtitude=2))
second_point = Place(name="End", position=Position(latitude=5, longtitude=100))

left_point = None
right_point = None

# Problem: make all this with one query


'''
    У нас есть эти точки изначально, поэтому можно вычислить их положение
'''
if first_point.position.longtitude <= second_point.position.longtitude: # 
    left_point = first_point
    right_point = second_point
else: #
    left_point = second_point
    right_point = first_point

query = f"SELECT * FROM Place WHERE ({left_point.position.latitude} >= {right_point.position.latitude} and (Place.longtitude <= {right_point.position.longtitude} and (Place.longtitude >= {left_point.position.longtitude} and Place.latitude <= {left_point.position.latitude} and Place.latitude >= {right_point.position.latitude})) or ({left_point.position.latitude} < {right_point.position.latitude} and (Place.longtitude <= {right_point.position.longtitude} and Place.longtitude >= {left_point.position.longtitude} and Place.latitude <= {right_point.position.latitude} and Place.latitude >= {left_point.position.latitude})));"
# print(query)
mycursor = mydb.cursor()
mycursor.execute(query)
myresult = mycursor.fetchall()
myresult

[(1, 'p_in_1', 2, 10.0, 50.0), (2, 'p_in_2', 2, 18.0, 70.0)]

In [36]:
first_point = Place(name="Start", position=Position(latitude=20, longtitude=2))
second_point = Place(name="End", position=Position(latitude=5, longtitude=100))
points = [first_point, second_point]

map_way = MapWay()
route = map_way.findBestWay(points=points)
for x in route:
    print(x.name)

Start
p_in_1
End


In [78]:
def primFindMST(G):
    import math, random, heapq, queue
    key, p, Q = [math.inf] * len(G) , [None] * len(G), FibonacciHeap_for_vertex()
    class Vertex:
        def __init__(self, key=None, v=None):
            self.key = key
            self.v = v
    r = random.randint(0, len(G) - 1)
    key[r] = 0
    for i in range(len(G)):
        mn = math.inf
        for j in range(len(G)):
            if G[i][j] < mn:
                mn = G[i][j]
                    
            mn = min(mn, G[i][j])
        vertex = Vertex(key=mn, v=i)
        Q.insert(vertex)
    heapq(heap, something) # something -> min(w(uv)) - вес минимального
    # ребра из вершин F в вершины G \ F
    while Q:
        v = heapq.heappop(heap)
#       Надо пройтись по всем ребрам графа
        for u in range(len(G[v])):
            if u in Q and key[u] > G[v][u]:
                p[u] = v
                key[u] = G[v][u]
                heapq.

False

In [99]:
from collections import defaultdict

INFINITY = float('inf')

""" For debugging purposes:
teststr = "silent"## Remember to use global when you use this.
def show(something):
	if teststr == "loud": print(something)
def struc(something):
	if teststr == "loud": something.print_heap()
"""


class FibHeapNode:
    """
    Each node contains
    - key aka ele
    - rank = no. of children
    - parent pointer because the node is a part of a heap ordered tree
    - left and right pointers because the node is a part of
    circular doubly linked list
    - mark is a boolean value to indicate whether node has lost a child
    """

    def __init__(self, ele):
        self.ele = ele
        self.rank = 0

        self.parent = None
        self.left = self
        self.right = self
        self.child = None

        self.mark = False  # True if node lost one child.

    def children_generator(self):
        # Generates children from the circular linked list self.child
        if self.child is None: return

        cur_child = self.child

        while cur_child.right is not self.child:
            yield cur_child
            cur_child = cur_child.right
        yield cur_child

    def print_tree(self, tabwidth=0):
        """
        Tabbed display of nodes
        No. of tabs = level at which that node is
        An asterisk after the value indicates that this node is marked
        """

        # if teststr == "silent":
        print(tabwidth * "    ", self.ele, '*' if self.mark else '', sep="")

        """ Debugging purposes
        elif teststr == "loud":
            print(tabwidth*"    ", end = " ")
            show((self.ele, id(self)))
        #input()#
        """
        for childtree in self.children_generator():
            childtree.print_tree(tabwidth + 1)

    def __str__(self):
        return str(self.ele)

    # Magic methods for comparison
    def __lt__(self, other):
        return self.ele < other.ele

    def __le__(self, other):
        return self.ele <= other.ele

    def __gt__(self, other):
        return self.ele > other.ele

    def __ge__(self, other):
        return self.ele >= other.ele

    def __eq__(self, other):
        return self.ele == other.ele


class FibHeap:
    """
    The root list is a circular doubly linked list.
    It's beginning is given by the head pointer.
    min_node points to the node with the minimum key.

    Each node in root list is the root of a heap ordered tree.
    """

    def __init__(self, head=None, min_node=None):
        if head is None:
            self.head = None
            self.min_node = None

        else:
            self.head = head
            self.min_node = min_node
        self.Node = FibHeapNode

    def merge(self, other):
        if self.head is None:
            self.head = other.head
            return
        elif other.head is None:
            return

        self.min_node = min(self.min_node, other.min_node)
        self.head = self._merge_lls(self.head, other.head)

        other.head = None
        other.min_node = None

    def __link(self, node1, node2):  # Assuming node1 and node2 are roots.
        node2.parent = node1
        node2.mark = False
        node1.rank += 1
        if node1.child:
            head = node1.child
            tail = head.left
            self.__attach(node2, head)
            self.__attach(tail, node2)

        else:
            node1.child = node2

    def __root_list_generator(self):
        if self.head is None: return
        cur_node = self.head.right

        while cur_node is not self.head:
            yield cur_node.left
            cur_node = cur_node.right

        yield cur_node.left

    def print_heap(self):
        print(f"### head = {self.head}, min_node = {self.min_node} ###")
        for root in self.__root_list_generator():
            root.print_tree()
        print()

    def _remove_node(self, node):
        # Caution: Updating min_node is not this method's concern
        # Assumes node is not None.
        if node is node.right:  # If heap only has one node right now.
            self.head = None
            return

        if self.head is node:  # If the head is the node to be popped.
            self.head = self.head.right

        self.__attach(node.left, node.right)
        node.left, node.right = node, node

    def _consolidate(self):
        self.degree_tree_map = defaultdict(lambda: None)

        def merging_trees(cur_root):
            other_root = self.degree_tree_map[cur_root.rank]

            if other_root is None:
                self.degree_tree_map[cur_root.rank] = cur_root
                return
            else:
                self.degree_tree_map[cur_root.rank] = None
                if cur_root <= other_root:
                    self._remove_node(other_root)
                    self.__link(cur_root, other_root)
                    combined_root = cur_root
                else:
                    self._remove_node(cur_root)
                    self.__link(other_root, cur_root)
                    combined_root = other_root

                merging_trees(combined_root)

        for cur_root in self.__root_list_generator():
            merging_trees(cur_root)

        # Maybe this can be done earlier.
        try:
            roots_iter = filter(lambda node: node is not None, self.degree_tree_map.values())
            self.min_node = min(roots_iter)
            """
            Note: This does not have to be the node of the first_occurence of the minimum ele
            since dictionaries are not ordered by their key.
            This means you could have a node with ele 4 as head and another node with ele 4 as tail
            and self.min_node could point to the latter.
            """
        except ValueError as err:
            if str(err) == "min() arg is an empty sequence":
                self.min_node = None
            else:
                raise ValueError(err)

    def __attach(self, node1, node2):
        # Connecting node 1 and node 2 such that node 2 is at node 1's right side
        node1.right = node2
        node2.left = node1

    def _merge_lls(self, head_one, head_two):
        # Merging two circular doubly linked lists and returning the new head.
        tail_one, tail_two = head_one.left, head_two.left
        self.__attach(tail_one, head_two)
        self.__attach(tail_two, head_one)
        return head_one

    def extract_min(self):
        if not self.head:
            raise IndexError("Popping from an empty heap.")

        node_to_be_popped = self.min_node

        if node_to_be_popped.child:
            # If the node to be popped has any children,
            # Add them to the root list.
            self._merge_lls(self.head, node_to_be_popped.child)

        self._remove_node(node_to_be_popped)
        # node_to_be_popped has now been popped.

        self._consolidate()

        temp = node_to_be_popped.ele
        node_to_be_popped.ele = None  # For later
        return temp

    def __cut(self, node):
        # Assumes node is not None and node.parent is not None.

        # print(f"Cutting {node}")
        if node is node.right:  # If cdll only has one node right now.
            node.parent.child = None

        else:
            if node.parent.child is node:  # If the head is the node to be popped.
                node.parent.child = node.parent.child.right

            self.__attach(node.left, node.right)

            node.left, node.right = node, node

        node.parent.rank -= 1
        node.parent = None

        self.__insert_node(node)

    def __cascading_cut(self, node):
        # print(f"Cascade Cutting {node}")
        # if node is a root
        if node.parent is None or node.parent.ele is None:
            pass

        else:
            if node.mark:
                parent = node.parent
                self.__cut(node)
                self.__cascading_cut(parent)
            else:
                node.mark = True

    def decrease_key(self, node, new_ele):
        if node.ele < new_ele:
            raise ValueError("new_ele is greater than node's current ele.")

        node.ele = new_ele

        """
        We don't have to change the position of node if 
        node is a root or if decrease key doesn't violate heap property.

        A node is in the root list because
        (i) It was inserted and an extract min hasn't occured since.
        (ii) It was inserted and when the extract min occured, it became a root of a tree.
        (iii) It was cut from its parent during decrease_key.
        (iv) It was on the first level and its parent was extracted, causing it to be added to the root list.

        For cases (i) and (ii), node.parent would be None.
        For case (iii), decrease_key's __cut clears node.parent
        For case (iv), node.parent.ele is set to infinity after extraction, 
        meaning node.ele would be lesser than node.parent.ele.
        """
        if node.parent is None or node.parent.ele is None:
            if self.min_node > node:
                self.min_node = node
            return

        elif node >= node.parent:
            pass
        else:
            parent = node.parent
            self.__cut(node)
            self.__cascading_cut(parent)

    def __insert_node(self, new_node):

        if self.head:
            self._merge_lls(self.head, new_node)

            if new_node < self.min_node: self.min_node = new_node

        else:  # empty heap
            self.head = new_node
            self.min_node = new_node

    def insert(self, new_ele):
        # Inserting the newnode at the end of the root list.
        new_node = self.Node(new_ele)
        self.__insert_node(new_node)


class FibHeapNodeForPrims(FibHeapNode):
    def __init__(self, vertex, ele):
        super().__init__(ele)
        self.vertex = vertex

    def __str__(self):
        return str((self.vertex, self.ele))

    def print_tree(self, tabwidth=0):
        """
        Tabbed display of nodes
        No. of tabs = level at which that node is
        An asterisk after the value indicates that this node is marked
        """

        # if teststr == "silent":
        print(tabwidth * "    ", self.vertex, ':', self.ele, '*' if self.mark else '', sep="")

        """ Debugging purposes
        elif teststr == "loud":
            print(tabwidth*"    ", end = " ")
            show((self.ele, id(self)))
        #input()#
        """
        for childtree in self.children_generator():
            childtree.print_tree(tabwidth + 1)


class FibHeapForPrims(FibHeap):
    def __init__(self, nfverts):  # Assumes nfverts is a natural no.

        # Every node needs to contain a vertex field.
        self.Node = FibHeapNodeForPrims

        src = 0

        # vertex_heapnode_map is implemented using a list instead of a dict
        # because vertices are labeled from 0 to nfverts - 1 anyway.
        self.vertex_heapnode_map = []

        head = self.Node(src, 0)
        tail = head
        self.vertex_heapnode_map.append(head)

        for vx in range(1, nfverts):
            node = self.Node(vx, INFINITY)
            self.vertex_heapnode_map.append(node)

            tail.right = node
            node.left = tail
            tail = node

        tail.right = head
        head.left = tail

        self.head = self.min_node = head

    def extract_min(self):
        if not self.head:
            raise IndexError("Popping from an empty heap.")

        node_to_be_popped = self.min_node

        if node_to_be_popped.child:
            # If the node to be popped has any children,
            # Add them to the root list.
            self._merge_lls(self.head, node_to_be_popped.child)

        self._remove_node(node_to_be_popped)
        # node_to_be_popped has now been popped.

        self._consolidate()

        node_to_be_popped.ele = None  # For later
        return node_to_be_popped.vertex

    def decrease_key(self, vertex, new_key):
        # Decreases the key for a given vertex to new_key.

        node = self.vertex_heapnode_map[vertex]
        super().decrease_key(node, new_key)

    def fetch_key(self, vertex):
        return self.vertex_heapnode_map[vertex].ele


class Graph:
    """
    We use a list of lists for adjacency matrix representation.
    We use a dictionary of dictionaries for adjacency lists representation.
    If vertex 0 is connected to vertex 4 with the weight 500
    and vertex 1 is connected to vertex 4 with the weight 900,
    this is what it looks like:
    {
    0: {4: 500},
    1: {4: 900},
    4: {0: 500, 1: 900}

    }

    In both representations, an edge weight can be accessed by graph[u][v]
    but the adj lists will raise a KeyError if there is no edge u-v.
    An object of this class cannot have both of these representations simulataneously.
    """

    def __init__(self, nfverts=None, representation=None):
        self.nfverts = nfverts  # no. of vertices

        self.representation = representation if representation else "matrix"
        if self.representation == "matrix":
            self.graph = []  # list of lists

        elif self.representation == "lists":
            self.graph = defaultdict(dict)  # dict of dicts

        else:
            raise ValueError("Invalid representation.")

    def read_from_file(self, input_file):
        # The input should be an adjacency matrix either way.

        self.nfverts = int(input_file.readline())

        if self.representation == "matrix":
            def inner_loop(u):
                self.graph.append([int(number) for number in input_file.readline().split()])
        else:
            def inner_loop(u):
                for v, number in enumerate(input_file.readline().split()):
                    number = int(number)
                    if number != 0: self.graph[u][v] = number

        for u in range(self.nfverts):
            inner_loop(u)

    def matrix_to_lists(self):
        if self.representation == "matrix":
            adj_lists = defaultdict(dict)
            for i, row in enumerate(self.graph):
                for j, ele in enumerate(row):
                    if ele != 0: adj_lists[i][j] = ele
            self.graph = adj_lists
            self.representation = "lists"

    def lists_to_matrix(self):
        if self.representation == "lists":
            adj_mat = []
            for i in range(self.nfverts):
                row = []
                for j in range(self.nfverts):
                    try:
                        ele = self.graph[i][j]
                    except KeyError:
                        ele = 0
                    row.append(ele)
                adj_mat.append(row)

            self.graph = adj_mat
            self.representation = "matrix"

    def fill_with_zeros(self):
        # Fill the adjacency matrix with zeros to initialize mst.

        assert self.representation == "matrix", "Wrong representation!"
        self.graph = [[0] * self.nfverts for _ in range(self.nfverts)]

    def __str__(self):
        if self.representation == "matrix":
            return "\n".join(str(row) for row in self.graph)
        else:  # You expect that it prints edges but that's not what it does.
            return '\n'.join(str(adj_list) for adj_list in self.graph.items())


def compute_mst_and_cost(precursor, grf):
    """
    Makes the adjacency matrix of the minimum spanning tree,
    returns that graph and also returns the total cost of the MST.
    """

    mst = Graph(grf.nfverts, representation="matrix")
    mst.fill_with_zeros()
    cost = 0

    # precursor[0] is useless.
    for cur, parent in enumerate(precursor[1:], 1):
        # print(f"(cur, par) = ({cur, parent})")
        mst.graph[parent][cur] = grf.graph[parent][cur]
        cost += grf.graph[parent][cur]

    return mst, cost

def prims_mst(grf):
    precursor = [None] * grf.nfverts
    visited = set()

    src = 0
    heap = FibHeapForPrims(grf.nfverts)

    for _ in range(grf.nfverts):
        u = heap.extract_min()
        # print("After extraction: \n"); heap.print_heap()
        visited.add(u)

        for v, cur_key in grf.graph[u].items():
            if v not in visited and cur_key < heap.fetch_key(v):
                heap.decrease_key(v, cur_key)
                # print(f"After decrease_key on {v}: \n"); heap.print_heap()
                precursor[v] = u

    return precursor

import math
g = Graph(representation='matrix', nfverts=6)
lst = [
    [math.inf, 3, 4, math.inf, 1],
    [3, math.inf, 5, math.inf, math.inf],
    [4, 5, math.inf, 2, 6],
    [math.inf, math.inf, 2, math.inf, 7],
    [1, math.inf, 6,  7, math.inf]
]
# lst = [
# [0, 4, 0, 0, 6, 5],
# [4, 0, 1, 0, 0, 4],
# [0, 1, 0, 6, 0, 4],
# [0, 0, 6, 0, 8, 5],
# [6, 0, 0, 8, 0, 2],
# [5, 4, 4, 5, 2, 0]
# ]
g.graph = lst
g.matrix_to_lists()
ans = prims_mst(g)
print(ans)

# print(lst)
# print(g.graph.items())

[None, 0, 0, 2, 0, None]
